In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Download the Data

In [2]:
# cpi = pd.read_csv(r'data/cpi.csv')
# fed_funds_rate = pd.read_csv(r'data/fed_funds_rate.csv')
# gdp = pd.read_csv(r'data/gdp.csv')
# sentiment = pd.read_csv(r'data/sentiment.csv')
# SP_price_action = pd.read_csv(r'data/SP_price_action.csv')
# u_rate = pd.read_csv(r'data/u_rate.csv')

# Clean the data
Get the data ready for a format to combine the data. Get the data and maybe some lag information.

In [3]:
# CPI 
cpi = pd.read_csv(r'data/cpi.csv')
cpi.columns = ['date', 'cpi_base']
cpi['date'] = pd.to_datetime(cpi['date'], format='%Y-%m-%d')
cpi['1yr_change'] = cpi['cpi_base'].diff()
cpi.head()

,date,cpi_base,1yr_change
0,1960-01-01,1.457976,NaN
1,1961-01-01,1.070724,-0.387252
2,1962-01-01,1.198773,0.128049
3,1963-01-01,1.239669,0.040896
4,1964-01-01,1.278912,0.039242


In [4]:
fed_funds_rate = pd.read_csv(r'data/fed_funds_rate.csv')
fed_funds_rate.columns = ["date", "base_fed_funds_rate"]
fed_funds_rate['date'] = pd.to_datetime(fed_funds_rate['date'], format='%Y-%m-%d')
fed_funds_rate['1day_fed_funds_change'] = fed_funds_rate['base_fed_funds_rate'].diff()
fed_funds_rate['1yr_fed_funds_diff'] = fed_funds_rate['base_fed_funds_rate'] - fed_funds_rate['base_fed_funds_rate'].shift(365) 
fed_funds_rate['1m_fed_funds_diff'] = fed_funds_rate['base_fed_funds_rate'] - fed_funds_rate['base_fed_funds_rate'].shift(30) 
fed_funds_rate

,date,base_fed_funds_rate,1day_fed_funds_change,1yr_fed_funds_diff,1m_fed_funds_diff
0,2019-06-06,2.37,NaN,NaN,NaN
1,2019-06-07,2.37,0.0,NaN,NaN
2,2019-06-08,2.37,0.0,NaN,NaN
3,2019-06-09,2.37,0.0,NaN,NaN
4,2019-06-10,2.37,0.0,NaN,NaN
...,...,...,...,...,...
1823,2024-06-02,5.33,0.0,0.25,0.0
1824,2024-06-03,5.33,0.0,0.25,0.0
1825,2024-06-04,5.33,0.0,0.25,0.0
1826,2024-06-05,5.33,0.0,0.25,0.0


In [5]:
gdp = pd.read_csv(r'data/gdp.csv')
gdp.columns = ["date", 'gdp_base']
gdp['date'] = pd.to_datetime(gdp['date'], format='%Y-%m-%d')
gdp = gdp[gdp['date'].dt.month == 1] # Came in by quarter, easiest thing to do was just take the first day of the year 
gdp['1yr_gdp_pct_change'] = gdp['gdp_base'].pct_change() * 100
gdp

,date,gdp_base,1yr_gdp_pct_change
0,1947-01-01,2182.681,NaN
4,1948-01-01,2239.682,2.611513
8,1949-01-01,2260.807,0.943214
12,1950-01-01,2346.104,3.772856
16,1951-01-01,2593.967,10.564877
...,...,...,...
292,2020-01-01,20665.553,1.226555
296,2021-01-01,20990.541,1.572607
300,2022-01-01,21738.871,3.565082
304,2023-01-01,22112.329,1.717927


In [6]:
sentiment = pd.read_csv(r'data/sentiment.csv')
sentiment.columns = ['date', 'base_sentiment']
sentiment = sentiment[sentiment['base_sentiment'] != '.']
sentiment['base_sentiment'] = sentiment['base_sentiment'].astype(float)

# There were a few months that were spread out and not one of them was the first of the year. I have taken the yearly average and placed that as the first of the year
sentiment['date'] = pd.to_datetime(sentiment['date'], format='%Y-%m-%d')
sentiment['year'] = sentiment['date'].dt.year
yearly_avg_sentiment = sentiment.groupby('year')['base_sentiment'].mean().reset_index()
yearly_avg_sentiment['date'] = pd.to_datetime(yearly_avg_sentiment['year'].astype(str) + '-01-01')
sentiment = yearly_avg_sentiment[['date', 'base_sentiment']]


sentiment['1yr_sentiment_pct_change'] = sentiment['base_sentiment'].pct_change() * 100

sentiment.head()

,date,base_sentiment,1yr_sentiment_pct_change
0,1952-01-01,86.200000,NaN
1,1953-01-01,84.066667,-2.474865
2,1954-01-01,83.966667,-0.118953
3,1955-01-01,98.233333,16.990869
4,1956-01-01,99.433333,1.221581


In [7]:
u_rate = pd.read_csv(r'data/u_rate.csv')
u_rate.columns = ['date', 'base_u_rate']
u_rate['date'] = pd.to_datetime(u_rate['date'], format='%Y-%m-%d')
u_rate['1m_u_rate_pct_change'] = u_rate['base_u_rate'].pct_change() * 100
u_rate['1yr_u_rate_pct_diff'] = u_rate['base_u_rate'].pct_change(periods=12) * 100

u_rate.head()

,date,base_u_rate,1m_u_rate_pct_change,1yr_u_rate_pct_diff
0,1948-01-01,3.4,NaN,NaN
1,1948-02-01,3.8,11.764706,NaN
2,1948-03-01,4.0,5.263158,NaN
3,1948-04-01,3.9,-2.500000,NaN
4,1948-05-01,3.5,-10.256410,NaN


# S&P 500 processing 
As this is the heart of our data and it is daily. For further analysis also I would want the data processed slightly differently. 

In [8]:
sp_price_action = pd.read_csv(r'data/SP_price_action.csv')
sp_price_action = sp_price_action[['Date', 'Adj Close', 'Volume']]
sp_price_action.columns = ['date', 'price', 'volume']
sp_price_action['date'] = pd.to_datetime(sp_price_action['date'])
sp_price_action = sp_price_action[~sp_price_action['price'].isna()]
sp_price_action = sp_price_action[sp_price_action['volume'] > 0]
sp_price_action.head()

,date,price,volume
5496,1950-01-03,16.66,1260000.0
5497,1950-01-04,16.85,1890000.0
5498,1950-01-05,16.93,2550000.0
5499,1950-01-06,16.98,2010000.0
5500,1950-01-09,17.08,2520000.0


## Adding columns to the data

In [9]:
#Columns to add
# Set 'date' as index
sp_price_action.set_index('date', inplace=True)

# Calculate 1-Day Price Difference
sp_price_action['price_1d_diff'] = sp_price_action['price'].diff()

# Calculate 1-Month Price Difference
sp_price_action['price_1m_diff'] = sp_price_action['price'] - sp_price_action['price'].shift(21)  # Approx. 21 trading days in a month

# Calculate 1-Day Volume Difference
sp_price_action['volume_1d_diff'] = sp_price_action['volume'].diff()

# Calculate 1-Month Volume Difference
sp_price_action['volume_1m_diff'] = sp_price_action['volume'] - sp_price_action['volume'].shift(21)  # Approx. 21 trading days in a month

# Calculate Percentage Change in Price
sp_price_action['price_pct_change'] = sp_price_action['price'].pct_change() * 100

# Calculate Rolling Mean Price (7-day)
sp_price_action['rolling_mean_price_7d'] = sp_price_action['price'].rolling(window=7).mean()

# Calculate Rolling Standard Deviation Price (7-day)
sp_price_action['rolling_std_price_7d'] = sp_price_action['price'].rolling(window=7).std()

# Calculate Rolling Mean Volume (7-day)
sp_price_action['rolling_mean_volume_7d'] = sp_price_action['volume'].rolling(window=7).mean()

# Calculate Rolling Standard Deviation Volume (7-day)
sp_price_action['rolling_std_volume_7d'] = sp_price_action['volume'].rolling(window=7).std()

# Calculate Price Momentum (14-day)
sp_price_action['momentum_14d'] = sp_price_action['price'] - sp_price_action['price'].shift(14)

# Calculate Exponential Moving Average (20-day)
sp_price_action['ema_20d'] = sp_price_action['price'].ewm(span=20, adjust=False).mean()

In [10]:
sp_price_action

,price,volume,price_1d_diff,price_1m_diff,volume_1d_diff,volume_1m_diff,price_pct_change,rolling_mean_price_7d,rolling_std_price_7d,rolling_mean_volume_7d,rolling_std_volume_7d,momentum_14d,ema_20d
date,,,,,,,,,,,,,
1950-01-03,16.660000,1.260000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.660000
1950-01-04,16.850000,1.890000e+06,0.190000,NaN,630000.0,NaN,1.140456,NaN,NaN,NaN,NaN,NaN,16.678095
1950-01-05,16.930000,2.550000e+06,0.080000,NaN,660000.0,NaN,0.474777,NaN,NaN,NaN,NaN,NaN,16.702086
1950-01-06,16.980000,2.010000e+06,0.050000,NaN,-540000.0,NaN,0.295334,NaN,NaN,NaN,NaN,NaN,16.728554
1950-01-09,17.080000,2.520000e+06,0.100000,NaN,510000.0,NaN,0.588928,NaN,NaN,NaN,NaN,NaN,16.762025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05,5354.029785,3.591460e+09,62.689941,173.289551,-116440000.0,-91790000.0,1.184765,5287.821359,36.580868,3.986640e+09,6.599526e+08,45.879883,5262.797892
2024-06-06,5352.959961,3.609990e+09,-1.069824,165.259766,18530000.0,-377900000.0,-0.019982,5294.524205,44.018139,3.966419e+09,6.704421e+08,55.859863,5271.384756
2024-06-07,5346.990234,3.692760e+09,-5.969727,159.320312,82770000.0,-149340000.0,-0.111522,5305.958496,46.012202,3.986420e+09,6.580175e+08,43.720214,5278.585277
